## 項目88 循環依存を取り除く方法を知る

他の人と協働していると、不可避的に、モジュール間で相互依存性が見つかるものです。単一のプログラムのさまざまな部分について独りで作業しているときにも、相互依存が生じることがあります。

例えば、GUI アプリケーションで、どこに文書を保存するか選ぶためのダイアログボックスを表示するとします。表示データは、イベントハンドラへの引数で指定できるでしょう。しかし、ダイアログを適切にレンダリングするためには、ユーザ設定などのグローバル状態を読み取っておく必要があります。

ここでは、グローバルな設定からデフォルトの文書保存場所を得るダイアログを定義します。


In [ ]:
# dialog.py
import app

class Dialog:
    def __init__(self, save_dir):
        self.save_dir = save_dir

save_dialog = Dialog(app.prefs.get('save_dir'))

def show():
    print('Showing the dialog!')

問題は、prefs オブジェクトを含む app モジュールが、 プログラム開始のダイアログを表示するために、dialog クラスをインポートしていることです。

In [ ]:
# app.py
import dialog

class Prefs:
    def get(self, name):
        pass

prefs = Prefs()
dialog.show()


これは、循環依存になっており、app モジュールをメインプログラムから次のように使おうとすると、例外が発生します。

In [ ]:
# main.py
import app

```python
Traceback (most recent call last):
  File "...\main.py", line 17, in <module>
    import app
  File "...\app.py", line 17, in <module>
    import dialog
  File "...\dialog.py", line 23, in <module>
    save_dialog = Dialog(app.prefs.get('save_dir'))
AttributeError: partially initialized module 'app' has no attribute 'prefs' (most likely due to a circular import)
```

ここで、何が起こっているかを理解するには、 Pythonの一般的なインポート機構を知る必要があります (詳細は組み込みパッケージ importlib 参照)。モジュールがインポートされたとき、Python が実際に行うことは、深さ順に次のようになります。

1. sys.path から始めてモジュールの位置を特定する([参考](https://note.nkmk.me/python-import-module-search-path/#syspath))。
2. モジュールからコードをロードして、コンパイルできることを確かめる。
3. 対応する空モジュールオブジェクトを作る。
4. モジュールを sys.module に挿入する。
5. モジュールオブジェクトのコードを実行して、内容を定義する。

循環依存の問題は、モジュールの属性が属性のためのコードが実行されるまで (ステップ5の後) 定義されないことです。しかし、モジュールは sys.module に挿入された直後 (ステップ4の後) に import 文でロードされるのです。

先ほどの例では、app モジュールが定義の前に dialog をインポートします。そして、dialog モジュールがappをインポートします。app はまだ実行が完了していない (dialog をインポート中) ので、app モジュールは空のシェル (ステップ4) のままです。すると、prefs を定義するコードがまだ実行していない (appのステップ5は終わっていない) ため、AttributeError が dialog のステップ5の途中で起こります。 

この問題に対する最適解は、コードをリファクタリングして、prefs のデータ構造が依存木の底に来るようにすることです。そうすると、app も dialog もともに同じユーティリティモジュールをインポートできて、循環依存を回避できます。しかし、そのようなすっきりした分割が常に可能とは限りませんし、労力の割に報われないリファクタリングの作業があまりにも多く必要とされるかもしれません。

循環依存を取り除くには、他に3つの方法があります。

### インポートの順序を変更する


第1の方式は、インポートの順序を変えることです。例えば、dialog モジュールを app モジュールの下のほうで、その内容を順番に実行した後でインポートすれば、AttributeError が解消します。


In [ ]:
# app.py ver2
class Prefs:
    def get(self, name):
        pass

prefs = Prefs()

import dialog  # Moved
dialog.show()

これがうまくいくのは、dialog モジュールが後でロードされたとき、その再帰的な app のインポートで app.prefs が既に定義されていること (app のステップ5がほぼ終了) がわかるからです。

これは、AttributeError を回避しますが、PEP 8 スタイルガイド (「項目2 PEP 8 スタイルガイドに従う」参照) に違反しています。スタイルガイドは、インポートを Python ファイルの先頭に置くよう指示しています。モジュールの依存関係が、コードの新たな読者にも明確になります。また、依存しているすべてのモジュールが、モジュールのすべてのコードのスコープにあって利用可能なことを保証します。

インポートがファイルの後ろのほうにあるということは、扱いにくく、コードの順番のちょっとした変更で、モジュール全体がダメになる危険性があります。したがって、循環依存の問題を解決するために、インポートの順番を変えるのは避けるべきです。


### インポート、構成、実行

循環インポート問題の第2の解法は、インポート時のモジュールの副作用を最小化することです。モジュールでは、関数、クラス、定数の定義だけを行います。インポート時には、関数を実際に実行しないようにします。そして、他のすべてのモジュールのインポートが終わったら、各モジュールが提供した configure 関数を実行します。configure の目的は、他のモジュールの属性にアクセスして、各モジュールの状態を準備しておくことにあります。すべてのモジュールがインポートされた (ステップ5が完了) 後で configure を実行するので、すべての属性が定義されています。

ここでは、dialog モジュールを再定義して、configure が呼ばれたときに、prefs オブジェクトだけにアクセスするようにします。


In [ ]:
# dialog.py ver3
import app

class Dialog:
    def __init__(self):
        pass

save_dialog = Dialog()

def show():
    print('Showing the dialog!')

def configure():
    save_dialog.save_dir = app.prefs.get('save_dir')

app モジュールも再定義して、インポートしたときに何も実行しないようにします。

In [ ]:
# app.py ver3
import dialog

class Prefs:
    def get(self, name):
        pass

prefs = Prefs()

def configure():
    pass

最後に、mainモジュールが、すべてをインポート、すべてを configure、最初のアクティビティを実行するという3段階を実行します。

In [ ]:
# main.py ver3
import app
import dialog

app.configure()
dialog.configure()

dialog.show()

これは、多くの状況でうまく働き、**依存注入（Dependency Injection: DI）** のようなパターンを可能にします。 しかし、時には、 明示的な configure ステップが可能となるようにコードを構造化することが困難なことがあります。2つの異なる段階をモジュールの中に持つのも、オブジェクトの定義をその構成から分離するために、コードを読みにくくしています。


### 動的インポート

第3の、しばしば最も簡単な循環インポート問題の解決法は、関数またはメソッドの中で import 文を使うことです。
これは、モジュールのインポートが、プログラムが最初に開始してモジュールを初期化しているときではなく、プログラムの実行中に起こるので、動的インポートと呼ばれます。

ここでは、dialogモジュールを再定義して動的インポートを使います。
dialog モジュールが初期化時に app をインポートするのではなくて、dialog.show 関数が実行時に app モジュールをインポートします。


In [ ]:
# dialog.py ver4

class Dialog:
    def __init__(self):
        pass

# Using this instead will break things
# save_dialog = Dialog(app.prefs.get('save_dir'))
save_dialog = Dialog()

def show():
    import app  # Dynamic import
    save_dialog.save_dir = app.prefs.get('save_dir')
    print('Showing the dialog!')


dialogを最初にインポートして dialog.show を最後に呼び出します (app モジュールは、元々の例にあったのと同じで構いません)。



In [ ]:
# app.py ver1
import dialog

class Prefs:
    def get(self, name):
        pass

prefs = Prefs()
dialog.show()


この方式は、1つ前の「インポート、構成、実行方式」と同様の効果があります。違いは、この方式がモジュールが定義されインポートされる方式に構造上の変更を必要としないことです。単に、循環インポートを他のモジュールにアクセスしなければならない瞬間まで遅らせるだけです。その時点では、他のすべてのモジュールが既に初期化されている (ステップ5がすべてで完了している) と考えられるでしょう。

ただし、一般にはこのような動的インポートは避けるべきです。import 文のコストは無視できませんし、ループでは特にひどく高価になりかねません。実行遅延によって、動的インポートは、実行時に、プログラムが開始してから随分時間が経ったのに、SyntaxError 例外が発生するなどという致命的なエラー (「項目76 関係する振る舞いをTestCase サブクラスで検証する」参照) を犯す危険もあります。しかし、これらの弱点も、プログラム全体を再構成するという別の解決策よりはましだという場合がしばしばあります。


### 覚えておくこと

- 循環依存は、2つのモジュールがインポート時に互いに呼び出すときに生じる。これは、プ
ログラムを開始時にクラッシュさせる。
- 循環依存を断ち切る最良の方法は、相互依存をリファクタリングして、依存木の底に切り離されたモジュールが来るようにすることである。
- 動的インポートが、 リファクタリングと複雑さを最小化して、モジュール間の循環依存を断ち切る最も単純な方式である。



## 項目 89 リファクタリングと利用のマイグレーションに warnings を考える

以前は予期されていなかったニーズを満たす新たな要求のために、API が変更されることはよくあります。API が小さくて、上にも下にも依存がわずかな間は、そのような変更は簡単です。プログラマが1人で、小さな API と、そのすべての呼び出し元を更新して、1回コミットすれば解決することはよくあります。

しかし、コードベースが大きくなると、API の呼び出し元が多くなったり、複数のソースリポジトリに分かれていて、API の変更で一度に呼び出し元を変更することが実際的でなかったり、不可能になってしまいます。その代わりに、協働してコードをリファクタリングし、API の使用法を最新の形式にマイグレーションしてくれるように、人々に注意して励ますための方法を考えないといけません。

例えば、自動車で平均速度と時間を指定し、どのくらいの距離を走行できるかを計算するモジュールを作るとします。つまり、平均速度 (マイル/時) と運転時間を引数として取る関数として定義します。


In [8]:
# Example 1
def print_distance(speed, duration):
    distance = speed * duration
    print(f'{distance} miles')

print_distance(5, 2.5)

12.5 miles


この関数はうまく動作し、この関数に依存するコードが多数あったとします。協働するプログラマは、このような距離を計算して出力する必要が共有コードベース全体であります。

現状うまくいっているようにも見えますが、このコードベースには、引数の距離の単位が明示されていないので間違いが起こりやすいという欠点があります。例えば、弾丸が秒速1,000メートルで3秒経ったときに、どこまで進んでいるか知ろうとしたときに、次のような間違った結果を得てしまうことがあり得ます。


In [9]:
# Example 2
print_distance(1000, 3)

3000 miles


この問題に対して、print_distance の API にオプションのキーワード引数 (「項目23 キーワード引数にオプションの振る舞いを与える」および「項目25 キーワード専用引数と位置専用引数で明確さを高める」参照) で speed, duration, 計算した距離の単位を指定できるようにします。

In [10]:
# Example 3
CONVERSIONS = {
    'mph': 1.60934 / 3600 * 1000,   # m/s
    'hours': 3600,                  # seconds
    'miles': 1.60934 * 1000,        # m
    'meters': 1,                    # m
    'm/s': 1,                       # m
    'seconds': 1,                   # s
}

def convert(value, units):
    rate = CONVERSIONS[units]
    return rate * value

def localize(value, units):
    rate = CONVERSIONS[units]
    return value / rate

def print_distance(speed, duration, *,
                   speed_units='mph',
                   time_units='hours',
                   distance_units='miles'):
    norm_speed = convert(speed, speed_units)
    norm_duration = convert(duration, time_units)
    norm_distance = norm_speed * norm_duration
    distance = localize(norm_distance, distance_units)
    print(f'{distance} {distance_units}')

弾丸の飛距離の関数呼び出しで、 正しい結果をマイルへの単位変換を指定するよう修正します。

In [11]:
# Example 4
print_distance(1000, 3,
               speed_units='meters',
               time_units='seconds')

1.8641182099494205 miles


この関数に単位指定を要求するのは良い方法に思えます。明示することでエラーの可能性が減り、初めての人にもコードが理解しやすくなります。しかし、API のすべての呼び出し元に、常に単位を指定するように移行することをどうすればできるでしょうか。 print_distance に依存するどのコードでも問題発生を最小限にして、呼び出し元が新たな単位引数をできるだけ早く採用するようにするにはどうすればいいでしょうか。

Python の組み込みモジュール warningsを使ってこの問題を解決します。warnings を使用する と、依存しているライブラリの変更に合わせてコードを修正するようにという通知を他のプログラムにするようプログラムできます。例外が主としてマシンによる自動エラー処理 (「項目87 APIからの呼び出し元を分離するために、ルート例外を定義する」 参照) であるのに対して、警告 (warnings) は、互いの協働で何を期待するかという人間間のコミュニケーションについてのものです。

print_distance を修正して、単位を指定するキーワード引数がないと警告を発するようにしま す。こうしておくと、引数はとりあえずオプションのままですが (「項目24 動的なデフォルト引数を 指定するときにはNoneとdocstring を使う」参照)、依存しているプログラムを実行している人々に、対応しないと将来問題が起きるかもしれないという注意を明示的に促すことができます。


In [12]:
# Example 5
import warnings

def print_distance(speed, duration, *,
                   speed_units=None,
                   time_units=None,
                   distance_units=None):
    if speed_units is None:
        warnings.warn(
            'speed_units required', DeprecationWarning)
        speed_units = 'mph'

    if time_units is None:
        warnings.warn(
            'time_units required', DeprecationWarning)
        time_units = 'hours'

    if distance_units is None:
        warnings.warn(
            'distance_units required', DeprecationWarning)  # 下記のwarning箇所
        distance_units = 'miles'

    norm_speed = convert(speed, speed_units)
    norm_duration = convert(duration, time_units)
    norm_distance = norm_speed * norm_duration
    distance = localize(norm_distance, distance_units)
    print(f'{distance} {distance_units}')

このコードが警告を発することを検証するため、関数を以前と同じ引数で呼び出し、warnings モジュールの sys.stderr 出力を捕捉します。


In [13]:
# Example 6
import contextlib
import io

fake_stderr = io.StringIO()
with contextlib.redirect_stderr(fake_stderr):
    print_distance(1000, 3,
                   speed_units='meters',
                   time_units='seconds')    # warning を出してほしいところ

print(fake_stderr.getvalue())


1.8641182099494205 miles
C:\Users\su10_\AppData\Local\Temp\ipykernel_23044\2134811440.py:19: DeprecationWarning: distance_units required
  warnings.warn(



この関数に警告を追加するには、多数の定型句を繰り返す必要があり、可読性も保守性も悪くなります。警告メッセージは、warnings.warn が呼ばれた行を示していますが、本当に表示したかったのは、もうすぐ必要になるキーワード引数なしに print_distance が呼ばれた箇所です。

幸い、warnings.warn 関数は stacklevel パラメータをサポートしており、警告の原因となったスタック上の正しい場所を報告することが可能です。stacklevel を使うと、定型句を省略して他のコードのための警告を発することも可能です。オプション引数がないと警告を発し、デフォルト値を与えるヘルパー関数を次のように定義します。


In [14]:
# Example 7
def require(name, value, default):
    if value is not None:
        return value
    warnings.warn(
        f'{name} will be required soon, update your code',
        DeprecationWarning,
        stacklevel=3)
    return default

def print_distance(speed, duration, *,
                   speed_units=None,
                   time_units=None,
                   distance_units=None):
    speed_units = require('speed_units', speed_units, 'mph')
    time_units = require('time_units', time_units, 'hours')
    distance_units = require(
        'distance_units', distance_units, 'miles')

    norm_speed = convert(speed, speed_units)
    norm_duration = convert(duration, time_units)
    norm_distance = norm_speed * norm_duration
    distance = localize(norm_distance, distance_units)
    print(f'{distance} {distance_units}')

捕捉した出力を調べて、 これが問題の行を正しく把握したか検証します。

In [15]:
# Example 8
import contextlib
import io

fake_stderr = io.StringIO()
with contextlib.redirect_stderr(fake_stderr):
    print_distance(1000, 3,
                   speed_units='meters',
                   time_units='seconds')

print(fake_stderr.getvalue())


1.8641182099494205 miles
C:\Users\su10_\AppData\Local\Temp\ipykernel_23044\1661391751.py:7: DeprecationWarning: distance_units will be required soon, update your code
  print_distance(1000, 3,



warnings モジュールでは、警告が発生したときの動作を設定することもできます。選択肢の1つとしては、すべての警告がエラーになることで、警告を sys.stderr に出力する代わりに例外として上げることです。

In [16]:
# Example 9
warnings.simplefilter('error')
try:
    warnings.warn('This usage is deprecated',
                  DeprecationWarning)
except DeprecationWarning:
    pass  # Expected
else:
    assert False

warnings.resetwarnings()

この例外を送出する振る舞いは、自動化テストにおいて、依存性の上流で変更を検出し、テスト時にエラーを送出させるので、特に役立ちます。このようなテスト時に警告を発生させることは、共同している人々にコード変更の必要性を明確にする非常に良い方法です。Python インタプリタで -W error コマンドライン引数を使うか、このオプションを省略するために環境変数 PYTHONWARNINGS を error 等の値に設定することでも適用できます。

```
$ python -W error example_test.py
Traceback (most recent call last):
File ".../example_test.py", line 6, in <module>
warnings.warn('This might raise an exception ! ' )
UserWarning: This might raise an exception!
```

非推奨APIに依存するコードの責任者が、マイグレーションの必要性に気付いたなら、warnings モジュールで simplefilter と filterwarnings 関数を用い、エラーを無視するように設定することもできます (詳細については https://docs.python.org/ja/3/library/warnings.html 参照)。

In [17]:
# Example 10
warnings.resetwarnings()

warnings.simplefilter('ignore')
warnings.warn('This will not be printed to stderr')

warnings.resetwarnings()

プログラムをプロダクションに使い出すと、警告からエラーを起こさせるのは、極めて重大な時期にプログラムをクラッシュさせるので、意味がなくなります。それよりも良い方式は、警告を組み込みモジュール logging に複製することです。次のコードでは、logging.captureWarnings 関数を呼び出し、対応する py.warnings ロガーを構成します。

In [18]:
# Example 11
import logging

fake_stderr = io.StringIO()
handler = logging.StreamHandler(fake_stderr)
formatter = logging.Formatter(
    '%(asctime)-15s WARNING] %(message)s')
handler.setFormatter(formatter)

logging.captureWarnings(True)
logger = logging.getLogger('py.warnings')
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

warnings.resetwarnings()
warnings.simplefilter('default')
warnings.warn('This will go to the logs output')

print(fake_stderr.getvalue())

warnings.resetwarnings()

2023-10-26 11:21:20,758 WARNING] C:\Users\su10_\AppData\Local\Temp\ipykernel_23044\212808671.py:17: UserWarning: This will go to the logs output
  warnings.warn('This will go to the logs output')




ロギングを使って警告を捕捉すると、プログラムで使っているエラー報告システムがプロダクションで重要な警告も受け取るようになることが確実になります。これは、テストがあらゆる極端な利用の場合を網羅しているわけではないため、特に有用です。

APIライブラリの保守担当者も正しい環境で明確で対応可能なメッセージを警告として出せていることを検証するユニットテストを検証すべきです (「項目76 関係する振る舞いを TestCase サブクラスで検証する」参照)。次のコードでは、warnings.catch_warnings 関数をコンテキストマネジャーとして使い (背景は「項目66 contextlib と with 文を try/finally の代わりに考える」参照) 上で定義 した require 関数の呼び出しをラップします。


In [19]:
# Example 12
with warnings.catch_warnings(record=True) as found_warnings:
    found = require('my_arg', None, 'fake units')
    expected = 'fake units'
    assert found == expected

警告メッセージを集めたら、番号、詳細メッセージ、カテゴリが自分の期待と合致しているか検証できます。

In [20]:
# Example 13
assert len(found_warnings) == 1
single_warning = found_warnings[0]
assert str(single_warning.message) == (
    'my_arg will be required soon, update your code')
assert single_warning.category == DeprecationWarning

### 覚えておくこと

- warnings モジュールは、APIの呼び出し元に非推奨利用について警告するのに使う。警告メッセージは、後でプログラムが変更により失敗する前に、 呼び出し元がコードを修正でき るようになる。
- -W error コマンドライン引数を使ってPython インタプリタに警告をエラーとして上げる。これは、依存性を減らす可能性を持つ自動化テストにおいて特に有用だ。
- プロダクションでは、警告を logging モジュールに複製して、既存のエラー報告システムが実行時に警告を捕捉することを確かにする。
- コードで起きる警告をテストして、依存する下流のコードで適切なときに警告が出されることを確かめる。


## 項目90 バグを回避するために静的解析を検討する

ドキュメントが提供されていると、API の利用者が正しい使い方を理解するのにとても役立ちますが (「項目84 すべての関数、クラス、モジュールについて docstring を書く」参照)、それだけでは十分でないことが多く、相変わらず不適切な使用によりバグを引き起こします。理想的には、呼び出し元がAPIを正しく使っていることと、API で下流での依存関係を正しく使っていること確認する仕組みがプログラムにあることです。多くのプログラミング言語で、この必要性に答えて、ある種のバグを検出して取り除くためにコンパイル時の型チェックを取り入れています。

歴史的に Python は、動的な機能に焦点を絞っており、いかなるコンパイル時の型安全性も提供しませんでした。しかし、最近 Python は、特別な構文と組み込みの typing モジュールを導入し、変数、クラスフィールド、関数、メソッドに型情報を与えられるようにしました。これらの型ヒントは、緩やかな型付けを可能にするとともに、望ましい型を緩やかに指定できるようにします。

Python プログラムに型情報を追加する利点は、静的解析ツールを実行してプログラムのソースコードに対して、どこにバグが潜んでいるかを検出できることです。組み込みモジュール typing 自体は、いかなる型チェック機能も実装していません。ジェネリクスを含めて型を定義するための共通ライブラリを提供しているだけです。これは、Python コードに適用して、別のツールでその結果を利用できます。

Python インタプリタには複数の異なる実装 (例: CPython、PyPy) があるので、typing を利用する静的解析ツールも複数の実装があります。本書執筆時点 (2019年秋) において、最もよく使われているのは、mypy (https://github.com/python/mypy)、pytype (https://github.com/google/pytype)、pyright (https://github.com/microsoft/pyright)、pyre (https://pyre-check.org) です。本書で取り上げた typing 事例には、--strict フラグで mypy を使いました。ツールで扱うすべての警告を扱います。コマンドラインの実行例を示します。



```shell-session
$ python3 -m mypy --strict example.py
```

このようなツールは、プログラム実行前に多数のよくあるエラーを捕捉します。これは、ユニットテストに加えて (項目76 関係する振る舞いを TestCase サブクラスで検証する」参照) 安全性のレイヤーを追加します。例えば、次の簡単な関数がコンパイルでは問題がなかったのに、実行時にエラー例外が発生するのは何故なのかわかるでしょうか。

```python code
def subtract (a,b):
    return a-b
    
subtract (10, '5')
```

```shell
>>>
Traceback ...
TypeError: unsupported operand type(s) for -: 'int' and 'str'

```

引数や変数の型ヒントは、name: type のようにコロンの後に指定します。戻り値の型は引数リストの後に -> type と書きます。型ヒントと mypy でバグが簡単に見つかります。

```python
# example.py
def subtract(a: int, b: int) -> int: # 関数の型ヒント
    return a - b

subtract(10, '5')   # おっと、文字列を渡した
```

```shell
$ python3 -m mypy --strict example.py
.../example.py:4: error: Argument 2 to "subtract" has
 incompatible type "str"; expected "int"
```

Python 2 から Python 3に移ってきたプログラマに特によくある間違いは、bytes と str のインスタンスを混在させて使うことです (項目3 bytes とstr の違いを知っておく」参照)。 実行時エラーを起こす次の例題の問題がわかるでしょうか。


In [10]:
def concat(a, b):
    return a + b

```python
concat('first', b'second')
```
```shell
>>>
Traceback ...
TypeError: can only concatenate str (not "bytes") to str
```

型ヒントとmypyを使うと、この問題がプログラムを実行する前に静的に検出できます。

```python
# example.py
def concat(a, b) -> str:    # 型ヒント
    return a + b

concat('first', b'second')  # おっと、bytes 値を渡した
```

```shell
$ python3 -m mypy --strict example.py
.../example.py:4: error: Argument 2 to "concat" has
 incompatible type "bytes"; expected "str"
```

型ヒントはクラスにも使えます。次のクラスにはバグが2つあり、実行時に例外が発生します。

In [11]:
class Counter:
    def __init__(self):
        self.value = 0

    def add(self, offset):
        value += offsetMode.noitsioo
   
    def get(self) -> int:
        self.value

add メソッド呼び出しで最初のエラーが起きます。

```python
counter = Counter()
counter.add(5)
```

```shell
>>>
Traceback ...
UnboundLocalError: local variable 'value' referenced before assignment
```

get を呼び出すと第2のエラーが起きます。

```python
counter = Counter()
found = counter.get()
assert found == 0, found
```
```shell
>>>
Traceback ...
AssertionError: None
```

両方の問題とも、mypy で簡単に見つかります。

```python
# example.py
class Counter:
    def __init__(self) -> None:
        self.value: int = 0 # フィールド/変数の型ヒント
    
    def add (self, offset: int) -> None:
        value += offset     # おっと。self.を忘れた
    
    def get(self) -> int:
        self.value          #おっと。return を忘れた

counter = Counter()
counter.add(5)
counter.add(3)
assert counter.get() == 8
```
```shell
$ python3 -m mypy --strict example.py
.../example.py:6: error: Name 'value' is not defined
.../example.py:8: error: Missing return statement
```



Pythonのダイナミズムの強みの1つが、 ダックタイピングを処理できるジェネリックな機能を書けるところです (項目15 dictの挿入順序に依存する場合は注意する」および「項目43 カスタムコンテナ型は collections.abc を継承する」参照)。これによって、1つの実装で広範囲の型を扱えて、重複作業が減り、テストが簡単になります。次のコードは、そのようなジェネリックな関数を定義して list の値を組み合わせます。 最後のアサーションが失敗するのはなぜかわかりますか。

In [12]:
def combine(func, values):
    assert len (values) > 0

    result = values[0]
    for next_value in values [1:]:
        result = func(result, next_value )
    
    return result

def add(x, y):
    return x + y

```python
inputs = [1, 2, 3, 4j]
result = combine(add, inputs)
assert result == 10, result # 失敗
```
```shell
Traceback ...
AssertionError: (6+4j)
```

typing モジュールのジェネリクスのサポートを使って、この関数の型ヒントを書き、問題を静的に検出できます。


In [13]:
from typing import Callable, List, TypeVar

Value = TypeVar('Value')
Func = Callable[[Value, Value], Value]

def combine(func: Func[Value], values: List[Value]) -> Value:
    assert len(values) > 0

    result = values[0]
    for next_value in values[1:]:
        result = func(result, next_value)

    return result

Real = TypeVar('Real', int, float)

def add(x: Real, y: Real) -> Real:
    return x + y


```python
inputs = [1, 2, 3, 4j]  # Oops: included a complex number
result = combine(add, inputs)
assert result == 10
```
```shell
$ python3 -m mypy --strict example.py
../example.py:21: error: Argument 1 to "combine" has
 incompatible type "Callable[[Real, Real], Real]"; expected
 "Callable [[complex, complex], complex]"
```

別の非常によくあるエラーが、正しいオブジェクトが得られるだろうと思っていたのに None 値が返されることでしょう (「項目20 None を返すのではなく例外を送出する」)。 この問題は、非常に簡単と思えるコードでも起こります。問題点がわかるでしょうか。


In [14]:
from typing import Optional

def get_or_default(value, default):
    if value is not None:
        return value
    return value

```python
found = get_or_default(3,5) 
assert found == 3

found = get_or_default(None, 5)
assert found = = 5, found   # 失敗
```
```shell
>>>
Traceback ...
AssertionError: None
```

typing モジュールは、**オプション型**をサポートして、適切な null チェックが行われた値とだけプログラムが扱うことを確実にします。これによって、mypy はコードにバグがあると推論できます。return 文に使われた型が None でなければならず、それは、関数シグネチャが要求する int 型と合致しません。

In [15]:
# example.py
from typing import Optional

def get_or_default(value: Optional[int],
                   default: int) -> int: 
    if value is not None:
        return value
    return value  # Oops: should have returned "default"

```shell
$ python3 -m mypy --strict example.py
.../example.py:7: error: Incompatible return value type (got "None",
 expected "int")
```

typing モジュールでは幅広くさまざまなオプションを使うことができます。 詳細については、 https://docs.python.org/ja/3/library/typing.html を参照してください。 例が含まれていないのは大きな特徴です。すべてのメソッドで API 境界を要求する例外をチェックする Java とは異なり、Python の型ヒントは、C# とよく似ています。例外は、インタフェースの定義の一部とは考えられていません。したがって、例外を送出し、捕捉することが適切か検証するためには、テストを書かねばなりません。

typing モジュールを使う上でよく問題になるのが、前方参照の扱いです (同様の問題は「項目88 循環依存を取り除く方法を知る」参照)。例えば、2つのクラスの片方が他を参照しているとします。

In [16]:
class FirstClass:
    def __init__(self, value):
        self.value = value

class SecondClass:
    def __init__(self, value):
        self.value = value

second = SecondClass(5)
first = FirstClass(second)

このプログラムに型ヒントを与えて mypy を実行すると問題は出ません。

In [17]:
# example.py
class FirstClass:
    def __init__(self, value: SecondClass) -> None: # うまくいかない
        self.value = value

class SecondClass:
    def __init__(self, value: int) -> None:
        self.value = value

second = SecondClass(5)
first = FirstClass(second)


```shell
$ python3 -m mypy --strict example.py
(-> ok)
```

しかし、実際に実行すると、`FirstClass.__init__` メソッドのパラメータの型ヒントで、
SecondClass が定義前に参照されているので、エラーが生じます。


```python
second = SecondClass(5)
first = FirstClass(second)
```
```shell
Traceback ...
NameError: name 'SecondClass' is not defined
```

静的解析ツールを使う回避法は、前方参照を含む型ヒントを文字列で指定することです。文字列値が後でパースされてチェックする型情報を評価抽出します。

In [18]:
class FirstClass:
    def __init__(self, value: SecondClass) -> None:  # OK
        self.value = value

class SecondClass:
    def __init__(self, value: int) -> None:
        self.value = value

second = SecondClass(5)
first = FirstClass(second)

より良い方式は、`from __future__ import annotations`を使うことです。これは Python 3.7 で導入されて、Python 3.10 ではデフォルトになる予定です。これを使うと、Python インタプリタは、型ヒントに指定された値をプログラム実行時に完全に無視します。これによって、前方参照の問題が解決し、プログラム開始時の性能が改善します。

In [19]:
from __future__ import annotations

class FirstClass:
    def __init__(self, value: SecondClass) -> None:  # OK
        self.value = value

class SecondClass:
    def __init__(self, value: int) -> None:
        self.value = value

second = SecondClass(5)
first = FirstClass(second)

型ヒントをどのように使うか、その便益がどのようなものかがわかったと思いますが、いつ使うかについてはよく考えることが重要です。覚えておくべきベストプラクティスを次に示します。

- 新たにコードを書くとき、最初から型ヒントを指定しようとすると作業が遅くなる。一般的な戦略は、最初のバージョンは型ヒントなしで書き、テストを書き、それから最も価値のある部分で 型ヒントを書くというものだ。
- 多数の呼び出し元 (したがって多数の人) が依存するAPIのような、コードベースの境界で、型ヒントが最も重要になる。型ヒントは、統合テスト (「項目77 setUp, tearDown, setUpModule, tearDownModule で他からテストを分離する」参照) や警告 (「項目89 リファク タリングと利用のマイグレーションに warningsを考える」参照)と相補う形で、コード変更に際 して、API呼び出し元が驚いたり、実行がうまくいかなくなることがないようにする。
- APIの部分以外では、コードベースで最も複雑で、エラーになりやすい箇所で型ヒントを使うの が有効だ。しかし、コード全体に型ヒントを指定するのは、労力の割に得るものが少ないのであまり価値がないだろう。
- 可能なら、静的解析をビルドとテストの自動システムに含めて、コードベースをコミットするたびに、エラーがないかチェックするようにすべきだ。さらに、型チェックに使う構成をリポジト リで保守管理し、協働するすべての人が同じ規則に従うようにすべきだ。
- コードに型情報を追加したなら、そのたびに型チェッカを実行することが重要だ。そうでないと、 型ヒントというスプリンクラーをあちらこちらに付けたはいいが、型チェックツールで膨大なエラーが出され、面倒な作業が発生するので、型ヒントをもう使いたくないと思うかもしれない。
  
最後に、多くの場合、型ヒントが必要なかったり、使いたくない場合もあることを認識しておくことが重要です。小さなプログラム、暫定的なコード、従来からのレガシーコードベース、プロトタイプなどでは、型ヒントにかける労力の割には、効果が見込めないものです。



### 覚えておくこと

- Python には、変数、フィールド、関数、メソッドに型情報を与える、特別な構文と組み込みのモジュールがある。 
- 静的型チェッカは型情報を使って、 実行時にエラーが生じる多くのよくあるバグを回避するのを助ける
- プログラムに型ヒントを採用し、API に使うには、さまざまなベストプラクティスがあるので、生産性を妨げないようにうまく使うべきだ。
